In [3]:
import os
import rasterio
import matplotlib.pyplot as plt
from imageio import get_writer, imread
from tqdm import tqdm

# Parameters
input_folder = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_with_time/daily_rasters_tif'  # Folder with GeoTIFFs
output_gif = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_with_time/water_fraction.gif'  # Output GIF
frame_duration = 0.1  # Duration of each frame in seconds
vmin, vmax = 0.0, 1.0  # Value range for colorbar

# Collect all GeoTIFF files
tif_files = sorted(
    [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.tif')]
)

# Create GIF
with get_writer(output_gif, mode='I', duration=frame_duration) as writer:
    for tif_file in tqdm(tif_files, desc="Creating GIF"):
        # Read GeoTIFF
        with rasterio.open(tif_file) as src:
            data = src.read(1)  # Read the first (and only) band
            transform = src.transform
        
        # Plot the raster data
        plt.figure(figsize=(8, 6))
        plt.imshow(
            data, cmap='turbo', vmin=vmin, vmax=vmax, extent=(
                transform.c, transform.c + transform.a * data.shape[1],
                transform.f + transform.e * data.shape[0], transform.f
            )
        )
        plt.colorbar(label="Water Fraction Predicted")
        plt.title(os.path.basename(tif_file))
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.tight_layout()

        # Save frame to a temporary file
        temp_img_path = "temp_frame.png"
        plt.savefig(temp_img_path, dpi=100)
        plt.close()

        # Read the saved PNG and ensure compatibility
        img = imread(temp_img_path)
        if img.dtype != 'uint8':
            img = (255 * (img / img.max())).astype('uint8')  # Normalize and convert to uint8

        # Append the image to the GIF
        writer.append_data(img)
        
        # Remove the temporary image
        os.remove(temp_img_path)

print(f"GIF created successfully: {output_gif}")

Creating GIF:   0%|                                                                          | 0/365 [00:00<?, ?it/s]/var/folders/75/726b68j900v0z8bv7slw21zm0000gt/T/ipykernel_5420/3090085651.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(temp_img_path)
Creating GIF: 100%|████████████████████████████████████████████████████████████████| 365/365 [01:03<00:00,  5.78it/s]


GIF created successfully: /Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_with_time/water_fraction.gif


In [7]:
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from tqdm import tqdm
from imageio import get_writer, imread
import os

# Parameters
input_folder = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_with_time/daily_rasters_tif'
output_gif = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_with_time/deviation_dynamics.gif'
lon_min, lon_max = 10, 40
lat_min, lat_max = 5, 20
resolution = 0.05
vmin, vmax = -0.5, 0.5  # Color range for deviation fraction visualization

# Generate grid coordinates
lon_grid = np.arange(lon_min, lon_max, resolution)
lat_grid = np.arange(lat_min, lat_max, resolution)
lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)

# Load all raster data to calculate median fraction
all_fractions = []
tif_files = sorted(
    [f for f in os.listdir(input_folder) if f.endswith('.tif')]
)

for tif_file in tqdm(tif_files, desc="Loading water fraction data"):
    with rasterio.open(f"{input_folder}/{tif_file}") as src:
        water_fraction = src.read(1)  # First band
        all_fractions.append(np.nan_to_num(water_fraction))

# Stack fractions to calculate median fraction
stacked_fractions = np.stack(all_fractions, axis=0)
median_fraction = np.median(stacked_fractions, axis=0)

# Generate GIF
with get_writer(output_gif, mode='I', duration=0.2) as writer:
    for i, tif_file in tqdm(enumerate(tif_files), total=len(tif_files), desc="Creating deviation GIF"):
        with rasterio.open(f"{input_folder}/{tif_file}") as src:
            water_fraction = src.read(1)  # First band

        # Calculate deviation from median
        deviation_fraction = water_fraction - median_fraction

        # Plot the deviation fraction
        plt.figure(figsize=(10, 6))
        plt.imshow(
            deviation_fraction,
            cmap='coolwarm',
            vmin=vmin,
            vmax=vmax,
            extent=(lon_min, lon_max, lat_min, lat_max),
        )
        plt.colorbar(label="Deviation Fraction")
        plt.title(f"Deviation Dynamics - {tif_file.split('_')[-1].split('.')[0]}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.tight_layout()

        # Save the frame as a PNG
        temp_img_path = "temp_frame.png"
        plt.savefig(temp_img_path, dpi=100)
        plt.close()

        # Read the saved PNG and ensure compatibility
        img = imread(temp_img_path)
        if img.dtype != 'uint8':
            img = (255 * (img / img.max())).astype('uint8')  # Normalize and convert to uint8

        # Append the image to the GIF
        writer.append_data(img)

        # Remove the temporary image
        os.remove(temp_img_path)

print(f"Deviation dynamics GIF created successfully: {output_gif}")

Creating deviation GIF:   0%|                                                                | 0/365 [00:00<?, ?it/s]/var/folders/75/726b68j900v0z8bv7slw21zm0000gt/T/ipykernel_5420/274459121.py:67: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(temp_img_path)
Creating deviation GIF: 100%|██████████████████████████████████████████████████████| 365/365 [01:22<00:00,  4.41it/s]


Deviation dynamics GIF created successfully: /Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_with_time/deviation_dynamics.gif


In [5]:
import os
import rasterio
import matplotlib.pyplot as plt
from imageio import get_writer, imread
from tqdm import tqdm

# Parameters
input_folder = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_without_location/daily_rasters_tif'  # Folder with GeoTIFFs
output_gif = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_without_location/water_fraction.gif'  # Output GIF
frame_duration = 0.1  # Duration of each frame in seconds
vmin, vmax = 0.0, 1.0  # Value range for colorbar

# Collect all GeoTIFF files
tif_files = sorted(
    [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.tif')]
)

# Create GIF
with get_writer(output_gif, mode='I', duration=frame_duration) as writer:
    for tif_file in tqdm(tif_files, desc="Creating GIF"):
        # Read GeoTIFF
        with rasterio.open(tif_file) as src:
            data = src.read(1)  # Read the first (and only) band
            transform = src.transform
        
        # Plot the raster data
        plt.figure(figsize=(8, 6))
        plt.imshow(
            data, cmap='turbo', vmin=vmin, vmax=vmax, extent=(
                transform.c, transform.c + transform.a * data.shape[1],
                transform.f + transform.e * data.shape[0], transform.f
            )
        )
        plt.colorbar(label="Water Fraction Predicted")
        plt.title(os.path.basename(tif_file))
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.tight_layout()

        # Save frame to a temporary file
        temp_img_path = "temp_frame.png"
        plt.savefig(temp_img_path, dpi=100)
        plt.close()

        # Read the saved PNG and ensure compatibility
        img = imread(temp_img_path)
        if img.dtype != 'uint8':
            img = (255 * (img / img.max())).astype('uint8')  # Normalize and convert to uint8

        # Append the image to the GIF
        writer.append_data(img)
        
        # Remove the temporary image
        os.remove(temp_img_path)

print(f"GIF created successfully: {output_gif}")

Creating GIF:   0%|                                                                          | 0/365 [00:00<?, ?it/s]/var/folders/75/726b68j900v0z8bv7slw21zm0000gt/T/ipykernel_5420/1497763143.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(temp_img_path)
Creating GIF: 100%|████████████████████████████████████████████████████████████████| 365/365 [01:19<00:00,  4.62it/s]


GIF created successfully: /Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_without_location/water_fraction.gif


In [8]:
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from tqdm import tqdm
from imageio import get_writer, imread
import os

# Parameters
input_folder = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_without_location/daily_rasters_tif'
output_gif = r'/Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_without_location/deviation_dynamics.gif'
lon_min, lon_max = 10, 40
lat_min, lat_max = 5, 20
resolution = 0.05
vmin, vmax = -0.5, 0.5  # Color range for deviation fraction visualization

# Generate grid coordinates
lon_grid = np.arange(lon_min, lon_max, resolution)
lat_grid = np.arange(lat_min, lat_max, resolution)
lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)

# Load all raster data to calculate median fraction
all_fractions = []
tif_files = sorted(
    [f for f in os.listdir(input_folder) if f.endswith('.tif')]
)

for tif_file in tqdm(tif_files, desc="Loading water fraction data"):
    with rasterio.open(f"{input_folder}/{tif_file}") as src:
        water_fraction = src.read(1)  # First band
        all_fractions.append(np.nan_to_num(water_fraction))

# Stack fractions to calculate median fraction
stacked_fractions = np.stack(all_fractions, axis=0)
median_fraction = np.median(stacked_fractions, axis=0)

# Generate GIF
with get_writer(output_gif, mode='I', duration=0.2) as writer:
    for i, tif_file in tqdm(enumerate(tif_files), total=len(tif_files), desc="Creating deviation GIF"):
        with rasterio.open(f"{input_folder}/{tif_file}") as src:
            water_fraction = src.read(1)  # First band

        # Calculate deviation from median
        deviation_fraction = water_fraction - median_fraction

        # Plot the deviation fraction
        plt.figure(figsize=(10, 6))
        plt.imshow(
            deviation_fraction,
            cmap='coolwarm',
            vmin=vmin,
            vmax=vmax,
            extent=(lon_min, lon_max, lat_min, lat_max),
        )
        plt.colorbar(label="Deviation Fraction")
        plt.title(f"Deviation Dynamics - {tif_file.split('_')[-1].split('.')[0]}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.tight_layout()

        # Save the frame as a PNG
        temp_img_path = "temp_frame.png"
        plt.savefig(temp_img_path, dpi=100)
        plt.close()

        # Read the saved PNG and ensure compatibility
        img = imread(temp_img_path)
        if img.dtype != 'uint8':
            img = (255 * (img / img.max())).astype('uint8')  # Normalize and convert to uint8

        # Append the image to the GIF
        writer.append_data(img)

        # Remove the temporary image
        os.remove(temp_img_path)

print(f"Deviation dynamics GIF created successfully: {output_gif}")

Creating deviation GIF:   0%|                                                                | 0/365 [00:00<?, ?it/s]/var/folders/75/726b68j900v0z8bv7slw21zm0000gt/T/ipykernel_5420/651687933.py:67: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(temp_img_path)
Creating deviation GIF: 100%|██████████████████████████████████████████████████████| 365/365 [01:27<00:00,  4.15it/s]


Deviation dynamics GIF created successfully: /Users/hyou/mnt/Public/CYGNSS_clip_csv/Merged_data/Random_forest_training_without_location/deviation_dynamics.gif
